In [2]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Loading in dataframe using API
url = "https://data.cityofnewyork.us/api/views/jr24-e7cr/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)

df.head()

<ipython-input-3-d1d04bdb71cc>:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,15396.82,GOV/NYC/068,Yes,128800.0,7387.97,216.0,2808.0,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14556.34,GOV/NYC/068,Yes,115200.0,6607.87,224.0,2912.0,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13904.98,GOV/NYC/068,Yes,103200.0,5919.55,216.0,2808.0,5177.43
3,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14764.04,GOV/NYC/068,Yes,105600.0,6057.22,208.0,2704.0,6002.82
4,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13729.54,GOV/NYC/068,Yes,97600.0,5598.34,216.0,2808.0,5323.20


In [4]:
# Column cleanup & data range (date)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df['revenue_month'] = pd.to_datetime(df['revenue_month'], errors='coerce')
print(df['revenue_month'].min(), df['revenue_month'].max())

2010-01-01 00:00:00 2024-12-01 00:00:00


In [5]:
# Renaming 'revenue_month' to 'period'
df.rename(columns={'revenue_month': 'period'}, inplace=True)

# Data cleaning for other fields
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

df['borough'] = df['borough'].str.upper().str.strip()
df = df[df['borough'].isin(['BROOKLYN', 'MANHATTAN', 'BRONX', 'QUEENS', 'STATEN ISLAND'])]
df = df[df['consumption_(kwh)'].notnull() & (df['consumption_(kwh)'] > 0)]
df = df.sort_values('period')

In [6]:
# Creating 'covid_phase' to categorize before, during, and after pandemic
df['covid_phase'] = pd.cut(
    df['period'],
    bins=[
        pd.to_datetime('2009-12-31'),  # before any data
        pd.to_datetime('2020-02-29'),  # end of pre-COVID
        pd.to_datetime('2021-12-31'),  # end of intense COVID period
        pd.to_datetime('2025-01-01')   # future
    ],
    labels=['Before', 'COVID', 'After'],
)

df['covid_phase'] = pd.Categorical(
    df['covid_phase'],
    categories=['Before', 'COVID', 'After'],
    ordered=True
)

In [7]:
# Checking entries per year
df['year'] = df['period'].dt.year
year_counts = df['year'].value_counts().sort_index()
print(year_counts)

year
2010    26139
2012    25936
2013    25901
2014    25532
2015    25729
2016    25656
2017    24429
2019    24169
2020    33703
2021    25459
2022    25243
2023    24955
2024    25088
Name: count, dtype: int64


In [8]:
# Get boroughs sorted by total consumption
borough_order = (
    df.groupby('borough')['consumption_(kwh)']
    .sum()
    .sort_values(ascending=False)
    .index.tolist()
)

# Apply that order as a categorical type
df['borough'] = pd.Categorical(df['borough'], categories=borough_order, ordered=True)

In [9]:
# Defining 'phase_summary'
phase_summary = df.groupby('covid_phase')[['consumption_(kwh)', 'current_charges']].sum().reset_index()
phase_summary.columns = ['COVID Phase', 'Total kWh', 'Total Cost ($)']

<ipython-input-9-3804ed08413f>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  phase_summary = df.groupby('covid_phase')[['consumption_(kwh)', 'current_charges']].sum().reset_index()


**Visuals For Pandemic Impact**

In [31]:
# Highest and Lowest Peaks in Cost During COVID

# Aggregate monthly electricity costs in millions
monthly_cost = (
    df.groupby('period', as_index=False)['kwh_charges']
    .sum()
    .assign(kwh_charges_millions=lambda x: x['kwh_charges'] / 1_000_000)
)

# Define masks for each phase
pre_covid = monthly_cost['period'] < '2020-03'
covid = (monthly_cost['period'] >= '2020-03') & (monthly_cost['period'] <= '2021-12')
post_covid = monthly_cost['period'] > '2021-12'

# Get peak and low points per phase
def get_extremes(df, phase_name):
    peak = df.nlargest(1, 'kwh_charges_millions').assign(label=f"{phase_name} Peak")
    low = df.nsmallest(1, 'kwh_charges_millions').assign(label=f"{phase_name} Low")
    return peak, low

pre_peak, pre_low = get_extremes(monthly_cost[pre_covid], "Pre-COVID")
covid_peak, covid_low = get_extremes(monthly_cost[covid], "COVID")
post_peak, post_low = get_extremes(monthly_cost[post_covid], "Post-COVID")

# Combine all annotations
all_points = pd.concat([pre_peak, pre_low, covid_peak, covid_low, post_peak, post_low])

# Initialize figure
fig = go.Figure()

# Line chart
fig.add_trace(go.Scatter(
    x=monthly_cost['period'],
    y=monthly_cost['kwh_charges_millions'],
    mode='lines',
    name='Electricity Cost',
    line=dict(color='teal')
))

# Annotate all peak and low points
for i, (_, row) in enumerate(all_points.iterrows()):
    is_peak = 'Peak' in row['label']
    color = 'crimson' if is_peak else 'navy'

    if is_peak:
        position = 'top right'
    else:
        # Cycle positions to reduce clutter
        position = ['bottom right', 'bottom center', 'bottom right'][i % 3]

    fig.add_trace(go.Scatter(
        x=[row['period']], y=[row['kwh_charges_millions']],
        mode='markers+text',
        marker=dict(size=8, color=color),
        text=[f"{row['period'].strftime('%b %Y')}: ${row['kwh_charges_millions']:.1f}M"],
        textposition=position,
        showlegend=False
    ))

# Add shaded region for COVID
fig.add_vrect(
    x0="2020-03-01", x1="2021-12-31",
    fillcolor="red", opacity=0.1,
    layer="below", line_width=0,
    annotation_text="COVID Phase", annotation_position="top left"
)

# Layout
fig.update_layout(
    title="Electricity Cost (KWH Charges Only) Peaks and Lows Across COVID Phases (2010–2024)",
    yaxis_title="Cost (Million $)",
    xaxis_title="Month & Year",
    template="simple_white",
    hovermode="x unified"
)

fig.show()

In [11]:
# Bar Chart

# Grouping and sorting
phase_borough_summary = (
    df.groupby(['covid_phase', 'borough'], as_index=False)['consumption_(kwh)']
    .sum()
)

# Plotting
fig = px.bar(
    phase_borough_summary,
    x='covid_phase',
    y='consumption_(kwh)',
    color='borough',
    barmode='group',
    text='consumption_(kwh)',
    title="Electricity Consumption by Borough and COVID Phase (NYCHA)",
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Format text and layout
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    yaxis_title="Total kWh (M = Million, G = Billion)",
    xaxis_title="COVID Phase",
    template="plotly_white",
    yaxis_tickformat=".2s",
    showlegend=True
)

fig.show()

<ipython-input-11-c26e2b8b8db5>:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [12]:
# Configuring
phase_borough = (
    df.groupby(['borough', 'covid_phase'], as_index=False)['consumption_(kwh)']
    .sum()
    .sort_values(['borough', 'covid_phase'])
)

pivot_df = phase_borough.pivot(index='borough', columns='covid_phase', values='consumption_(kwh)').reset_index()

# Calculating Percentage Changes
pivot_df['% Change COVID vs Before'] = (pivot_df['COVID'] - pivot_df['Before']) / pivot_df['Before'] * 100
pivot_df['% Change After vs COVID'] = (pivot_df['After'] - pivot_df['COVID']) / pivot_df['COVID'] * 100

# Reshaping
melted = pivot_df.melt(
    id_vars='borough',
    value_vars=['% Change COVID vs Before', '% Change After vs COVID'],
    var_name='Phase Transition',
    value_name='Percent Change'
)

# Plotting
fig = px.bar(
    melted,
    x='borough',
    y='Percent Change',
    color='Phase Transition',
    barmode='group',
    text='Percent Change',
    title='Percent Change in Electricity Consumption by Borough Across COVID Phases',
    color_discrete_sequence=px.colors.qualitative.Set2
)
# Axis layout
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    yaxis_title="Percent Change (%)",
    xaxis_title="Borough",
    template="plotly_white",
    showlegend=True
)

# Reference Line at 0%
fig.add_shape(
    type="line",
    x0=-0.5, x1=len(pivot_df['borough']) - 0.5,
    y0=0, y1=0,
    line=dict(color="black")
)

fig.show()

<ipython-input-12-32e644765720>:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

